In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

newspaper_df = pd.read_excel('./data/NewspaperChurn new version.xlsx')

# 'Age range'가 null인 row 제거
newspaper_df = newspaper_df.dropna(subset=['Age range'])

age_group_bounds = {
    '24 years or less': (18, 24), # 24세 이하 (최소값은 임의로 18로 설정)
    '25-29': (25, 29),
    '30-34': (30, 34),
    '35-39': (35, 39),
    '40-44': (40, 44),
    '45-49': (45, 49),
    '50-54': (50, 54),
    '55-59': (55, 59),
    '60-64': (60, 64),
    '65-69': (65, 69),
    '70-74': (70, 74), 
    '75 years or more': (75, 80) # 나이 최대 값은 80세
}

# 나이 부여 함수
def sample_age(age_range, n_samples):
    low, high = age_group_bounds[age_range]
    mean = (low + high) / 2  # 평균은 범위의 중앙값
    std = (high - low) / 4   # 표준편차는 범위의 1/4 정도로 설정 (약 95%가 범위 안에 들도록)
    sampled = np.random.normal(loc=mean, scale=std, size=n_samples) # 정규분포 샘플링
    return np.clip(np.round(sampled), low, high).astype(int) # 정수로 반올림하고 범위 내로 제한


# Age 컬럼 생성
newspaper_df['Age'] = newspaper_df['Age range'].apply(
    lambda group: sample_age(group, 1)[0] if pd.notnull(group) else np.nan)


# 1. 소득 구간별 하한/상한 정의
income_bounds = {
    'Under $20,000': (10000, 19999),
    '$  20,000 - $29,999': (20000, 29999),
    '$  30,000 - $39,999': (30000, 39999),
    '$  40,000 - $49,999': (40000, 49999),
    '$  50,000 - $59,999': (50000, 59999),
    '$  60,000 - $69,999': (60000, 69999),
    '$  70,000 - $79,999': (70000, 79999),
    '$  80,000 - $89,999': (80000, 89999),
    '$  90,000 - $99,999': (90000, 99999),
    '$100,000 - $124,999': (100000, 124999),
    '$125,000 - $149,999': (125000, 149999),
    '$150,000 - $174,999': (150000, 174999),
    '$175,000 - $199,999': (175000, 199999),
    '$200,000 - $249,999': (200000, 249999),
    '$250,000 - $299,999': (250000, 299999),
    '$300,000 - $399,999': (300000, 399999),
    '$400,000 - $499,999': (400000, 499999),
    '$500,000 Plus': (500000, 750000),  # 상위 1% 소득 하한($631,500)을 감안해서 유연함 반영 (출처: DQYDJ, 2024)
}

# 2. 로그 정규분포 기반 샘플링 함수 정의
def sample_log_normal_income(income_range, n_samples=1):
    if income_range not in income_bounds or pd.isnull(income_range):
        return np.nan

    low, high = income_bounds[income_range]
    mu = np.log((low + high) / 2)
    sigma = 0.4  # 분포의 퍼짐 정도 (값이 작을수록 중앙에 집중, 클수록 상한/하한 근처까지 다양하게 퍼짐 => 0.4, 0.5가 중간값 중심 분포를 유지하면서도 현실적인 다양성을 부여하는 수준)

    sample = np.random.lognormal(mean=mu, sigma=sigma, size=n_samples)
    clipped = np.clip(sample, low, high)
    return int(round(clipped[0]))

# 3. 적용: 새로운 'Income' 컬럼 생성
newspaper_df['Income'] = newspaper_df['HH Income'].apply(sample_log_normal_income)

newspaper_df = newspaper_df.dropna(subset=['Nielsen Prizm'])


# 예시: Subscirber==0  =  is_churned==1 
newspaper_df['is_churned'] = newspaper_df['Subscriber'].apply(lambda x: 1 if x == 'NO' else 0)

# weekly fee 결측 여부를 나타내는 새로운 컬럼 생성 (1 = 결측, 0 = 비결측)
newspaper_df['weekly_fee_missing'] = newspaper_df['weekly fee'].isnull().astype(int)


In [2]:
newspaper_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15622 entries, 0 to 15854
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SubscriptionID      15622 non-null  int64 
 1   HH Income           15622 non-null  object
 2   Home Ownership      15622 non-null  object
 3   Ethnicity           15622 non-null  object
 4   dummy for Children  15622 non-null  object
 5   Year Of Residence   15622 non-null  int64 
 6   Age range           15622 non-null  object
 7   Language            14628 non-null  object
 8   Address             15622 non-null  object
 9   State               15622 non-null  object
 10  City                15622 non-null  object
 11  County              15622 non-null  object
 12  Zip Code            15622 non-null  int64 
 13  weekly fee          15438 non-null  object
 14  Deliveryperiod      15622 non-null  object
 15  Nielsen Prizm       15622 non-null  object
 16  reward program      15622 n

In [3]:
# dummy for Children, weekly_fee_missing 제외, weeklyfee의 결측치 제거
newspaper_df = newspaper_df.drop(columns=['dummy for Children', 'Subscriber', 'weekly_fee_missing', 'Age range', 'HH Income', 'SubscriptionID', 'State', 'Zip Code', 'Address'])
newspaper_df = newspaper_df.dropna(subset=['weekly fee'])
newspaper_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15438 entries, 0 to 15854
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Home Ownership     15438 non-null  object
 1   Ethnicity          15438 non-null  object
 2   Year Of Residence  15438 non-null  int64 
 3   Language           14461 non-null  object
 4   City               15438 non-null  object
 5   County             15438 non-null  object
 6   weekly fee         15438 non-null  object
 7   Deliveryperiod     15438 non-null  object
 8   Nielsen Prizm      15438 non-null  object
 9   reward program     15438 non-null  int64 
 10  Source Channel     15438 non-null  object
 11  Age                15438 non-null  int64 
 12  Income             15438 non-null  int64 
 13  is_churned         15438 non-null  int64 
dtypes: int64(5), object(9)
memory usage: 1.8+ MB


In [4]:
cat_cols = newspaper_df.select_dtypes(include=['object']).columns
newspaper_df_encoded = pd.get_dummies(newspaper_df, columns=cat_cols, drop_first=False)
newspaper_df_encoded.head()

,Year Of Residence,reward program,Age,Income,is_churned,Home Ownership_OWNER,Home Ownership_RENTER,Ethnicity_African (other),Ethnicity_African American 95%+,Ethnicity_Albanian,...,Source Channel_RetenOut,Source Channel_SCINSRT4,Source Channel_SCINSRT6,Source Channel_SCinsert,Source Channel_System,Source Channel_TMC,Source Channel_TeleIn,Source Channel_TeleOut,Source Channel_VRU,Source Channel_iSrvices
0,1,0,27,39999,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,14,0,50,750000,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,7,0,47,100000,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,23,1,56,249999,1,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,23,0,60,59999,0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
from sklearn.preprocessing import StandardScaler

# 1) 수치형 컬럼 선택
num_cols = newspaper_df_encoded.select_dtypes(include=['int64', 'float64']).columns

# 2) StandardScaler 적용
scaler = StandardScaler()
newspaper_df_encoded[num_cols] = scaler.fit_transform(newspaper_df_encoded[num_cols])

# 3) 확인
newspaper_df_encoded.head()


,Year Of Residence,reward program,Age,Income,is_churned,Home Ownership_OWNER,Home Ownership_RENTER,Ethnicity_African (other),Ethnicity_African American 95%+,Ethnicity_Albanian,...,Source Channel_RetenOut,Source Channel_SCINSRT4,Source Channel_SCINSRT6,Source Channel_SCinsert,Source Channel_System,Source Channel_TMC,Source Channel_TeleIn,Source Channel_TeleOut,Source Channel_VRU,Source Channel_iSrvices
0,-1.067116,-0.115900,-1.632169,-0.641693,0.491524,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0.046968,-0.115900,-0.158107,5.680316,-2.034490,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,-0.552923,-0.115900,-0.350376,-0.107431,-2.034490,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0.818257,-0.012403,0.226431,1.228194,0.491524,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0.818257,-0.115900,0.482790,-0.463609,-2.034490,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
display(newspaper_df_encoded)
display(newspaper_df_encoded.info())

,Year Of Residence,reward program,Age,Income,is_churned,Home Ownership_OWNER,Home Ownership_RENTER,Ethnicity_African (other),Ethnicity_African American 95%+,Ethnicity_Albanian,...,Source Channel_RetenOut,Source Channel_SCINSRT4,Source Channel_SCINSRT6,Source Channel_SCinsert,Source Channel_System,Source Channel_TMC,Source Channel_TeleIn,Source Channel_TeleOut,Source Channel_VRU,Source Channel_iSrvices
0,-1.067116,-0.115900,-1.632169,-0.641693,0.491524,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0.046968,-0.115900,-0.158107,5.680316,-2.034490,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,-0.552923,-0.115900,-0.350376,-0.107431,-2.034490,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0.818257,-0.012403,0.226431,1.228194,0.491524,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0.818257,-0.115900,0.482790,-0.463609,-2.034490,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15849,-0.895718,-0.115900,0.162341,-0.778427,0.491524,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15850,-0.895718,-0.115900,-1.696258,-0.196482,0.491524,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15851,-0.895718,-0.115900,-1.632169,-0.208262,0.491524,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15852,-0.124429,-0.115900,0.610969,0.560377,0.491524,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


<class 'pandas.core.frame.DataFrame'>
Index: 15438 entries, 0 to 15854
Columns: 278 entries, Year Of Residence to Source Channel_iSrvices
dtypes: bool(273), float64(5)
memory usage: 4.7 MB


None

In [7]:
newspaper_df_encoded['Age']

0       -1.632169
1       -0.158107
2       -0.350376
3        0.226431
4        0.482790
           ...   
15849    0.162341
15850   -1.696258
15851   -1.632169
15852    0.610969
15854    1.187776
Name: Age, Length: 15438, dtype: float64

In [8]:
newspaper_df_encoded['Income']

0       -0.641693
1        5.680316
2       -0.107431
3        1.228194
4       -0.463609
           ...   
15849   -0.778427
15850   -0.196482
15851   -0.208262
15852    0.560377
15854    0.782983
Name: Income, Length: 15438, dtype: float64

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier

# y는 원본 df(스케일 전)에서
y = newspaper_df['is_churned']

# X는 인코딩/스케일된 프레임에서 타깃을 제외
X = newspaper_df_encoded.drop(columns=['is_churned'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=0
)

from xgboost import XGBClassifier
xgb_clf = XGBClassifier(
    n_estimators=300, max_depth=6, learning_rate=0.1,
    subsample=0.8, colsample_bytree=0.8,
    eval_metric='logloss', tree_method='hist', n_jobs=-1, random_state=0
)
xgb_clf.fit(X_train, y_train)


from sklearn.metrics import precision_score, recall_score, f1_score, r2_score

# 예측
y_pred_train = xgb_clf.predict(X_train)
y_pred_test = xgb_clf.predict(X_test)


print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_test, y_pred_test))

print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

0.9117408906882591
0.8584844559585493
[[ 286  315]
 [ 122 2365]]
              precision    recall  f1-score   support

           0       0.70      0.48      0.57       601
           1       0.88      0.95      0.92      2487

    accuracy                           0.86      3088
   macro avg       0.79      0.71      0.74      3088
weighted avg       0.85      0.86      0.85      3088



In [10]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression()),
        ('dt', DecisionTreeClassifier()),
        ('svc', SVC(probability=True))
    ],
    final_estimator=LogisticRegression(),  # 메타 모델
    cv=5  # 교차 검증
)
stacking_clf.fit(X_train, y_train)
print("Stacking Accuracy:", stacking_clf.score(X_test, y_test))

Stacking Accuracy: 0.8516839378238342


---

In [23]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix, classification_report)



smote = SMOTE(random_state=42)
X_resample, y_resample = smote.fit_resample(X, y)

print(f'After SMOTE : { np.bincount(y_resample) }')
print(X_resample.shape)


After SMOTE : [12434 12434]
(24868, 277)


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, random_state=0)
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90      3057
           1       0.90      0.91      0.90      3160

    accuracy                           0.90      6217
   macro avg       0.90      0.90      0.90      6217
weighted avg       0.90      0.90      0.90      6217

